# Ingress Services Lab

### Introduction

In this lesson, we'll use what we learned about ingress services to allow external access to our cluster.  Let's get started.

### Setting it Up

Now, if you have not yet installed the ingress-nginx service, now is a good time.  To do so, we first run `minikube addons list` which should show us the list of addons.  And then run `minikube addons enable ingress`.

Now this will install a number of resources in the kube-system namespace.  So we can see this by viewing the pods now available in the `kube-system` namespace.  Run the following to take a look:

`kubectl get pods -n kube-system`

> <img src="./show_ingress.png" width="60%">

Notice that we now see the `ingress-nginx-controller` -- this is the nginx software that we will need to configure to route requests to our services.  

Importantly, with our nginx-controller installed, we can now see that if we visit the ip address of our cluster, we'll now see this points to nginx.  We can view the ip address of our cluster by visiting `minikube ip` and then going to our ip address.

> <img src="./nginx_404.png" width="60%">

Next, remember that we'll setup our `/etc/hosts` file so that a request to a domain name of `apartments.com` goes to our cluster's ip address.

> `sudo vim /etc/hosts`

Then configure the file appropriately.  Now, if we visit `apartments.com` we should see the same nginx screen.  

> **Warning**: We may have to view the domain while in incognito mode, as otherwise our browser may rely on a cached request. 

### Adding to our ingress

Ok, now it's time to add an ingress.  We'll do so such that we expose endpoints that point to the `frontend-service` and the `backend-service`.  The endpoint of the frontend service is root url -- `/`, and the endpoint of the backend service should be `apartments.com/api`.

> We'll provide the frontend service for you, which you simply need to add to a yaml file, and apply the changes.

```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
    name: public-routing
spec:
    rules:
        - host: apartments.com
          http:
            paths:
                - path: /
                  pathType: Prefix
                  backend: 
                    service:
                      name: frontend-service
                      port: 
                        number: 8501
```

Then confirm that you can access the streamlit service by visiting the ip address of the cluster.

> It may take a number of minutes to boot up, check the `get pods` and `describe pods` command to check the progress. 

> <img src="./working-application.png" width="40%">

**A quick fix**: The frontend service is of type NodePort.  As we know, this makes the service publicly accessible.  Update the service so that it is only accessible from within the cluster, and check that our website is still working.  We'll also want to change the backend service from type NodePort so that the service is not directly accessible.  

### Exposing the Backend Service

Ok, now let's add a route to the backend service.

We'll want any requests to `/api` to be transferred to the backend service, so configure the ingress so that a request to `/api` points to the backend service.  This, in theory, will allow us to visit `apartments.com/api/apartments` and see the corresponding json.

> If you see a 404 to a flask backend, you are on the correct track.  We'll explain how to proceed further below.

> <img src="./proper_404.png" width="80%">

The above is the 404 for the flask application.  We can get a sense of what's occurring by looking at the logs for our backend pods.  Run the following to view the logs, and you may see something like the following.

<img src="./problem_with_api.png" width="80%">

So notice above that the request is coming into our api pods as `/api/apartments`.  And the issue is that currently, our apartments json is accessible at simply `/apartments`.  So let's update our backend service in the section below.

### Updating the images

To make the update, we'll have to change the endpoint in flask, so that we now make a request to `/api/apartments` and are returned the json.

> <img src="./working_json.png" width="100%">

So while our backend is now accessible at `/api/apartments` if we view our frontend image, we will likely find it to be broken.  Update the frontend image so that it is making requests to the proper endpoint, and then confirm that we can access a working frontend and backend application.

### Resources
[Debug Services](https://kubernetes.io/docs/tasks/debug-application-cluster/debug-service/)

[K8s Workflow](https://www.getambassador.io/resources/dev-workflow-intro/)

[Telepresence](https://www.telepresence.io/tutorials/kubernetes-rapid)

